In [19]:
#package installs
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import unidecode
import re

In [20]:
def statsParser(name, year, playerindex, team_id):
    index = int(playerindex)
    lname = name[1]
    fname = name[0]
    lnamelist = list(lname)
    linitial = lnamelist[0]
    fnamelist = list(fname)
    lname5 = ''
    for x in range(5):
        try:
            lname5 += lnamelist[x]
        except:
            break
    f2i = fnamelist[0] + fnamelist[1]
    url = f'https://www.basketball-reference.com/players/{linitial}/{lname5}{f2i}{playerindex}/gamelog/{year}/'
    print(url)
    try:
        result = requests.get(url)
        print(result.status_code)
        src = result.content
        soup = BeautifulSoup(src, 'lxml')
        columns = ['G', 'Date', 'Age', 'Tm', 'Opp', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', '', '']
        table = soup.find('table', {'class': 'row_summable'})
        trs = table.find_all('tr')
        rows = []
        for tr in trs:
            tds = tr.find_all('td')
            row = [td.text.replace('\n', '').strip() for td in tds]
            rows.append(row)
        df = pd.DataFrame(rows, columns=columns)
        df = df[df['Tm'] == team_id]
        if len(df) == 0:
            index += 1
            newindex = '0' + str(index)
            statsParser(name, year, newindex, team_id)
        result.close()
        df['PlayerID'] = name[2]
        return df
    except:
        index+=1
        if index > 4:
            print('Stat parsing failed for:', fname, lname)
            print(url)
            return None
        newindex = '0' + str(index)
        statsParser(name, year, newindex, team_id)

In [21]:
def advancedStatsParser(name, year, playerindex, team_id):
    index = int(playerindex)
    lname = name[1]
    fname = name[0]
    lnamelist = list(lname)
    linitial = lnamelist[0]
    fnamelist = list(fname)
    lname5 = ''
    for x in range(5):
        try:
            lname5 += lnamelist[x]
        except:
            break
    f2i = fnamelist[0] + fnamelist[1]
    url = f'https://www.basketball-reference.com/players/{linitial}/{lname5}{f2i}{playerindex}/gamelog-advanced/{year}/'
    print(url)
    #try:
    result = requests.get(url)
    print(result.status_code)
    src = result.content
    soup = BeautifulSoup(src, 'lxml')
    columns = ['G', 'Date',	'Age', 'Tm', 'Opp', 'GS', 'MP', 'TS%', 'eFG%', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'GmSc', 'BPM', '', '']
    table = soup.find('table', {'class': 'row_summable'})
    trs = table.find_all('tr')
    rows = []
    for tr in trs:
        tds = tr.find_all('td')
        row = [td.text.replace('\n', '').strip() for td in tds]
        rows.append(row)
    df = pd.DataFrame(rows, columns=columns)
    df = df[df['Tm'] == team_id]
    if len(df) == 0:
        index += 1
        newindex = '0' + str(index)
        advancedStatsParser(name, year, newindex, team_id)
    result.close()
    df = df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6]], axis=1)
    print(df)
    return df
    #except:
        #index+=1
        #if index > 4:
            #print('Stat parsing failed for:', fname, lname)
            #print(url)
            #return None
        #newindex = '0' + str(index)
        #advancedStatsParser(name, year, newindex, team_id)

In [22]:
def concatStats(df1, df2):
    df = df1.join(df2)
    return df

In [23]:
def parse_roster(TeamID, year, id_number):
    url = f"https://www.basketball-reference.com/teams/{TeamID}/{year}.html"
    result = requests.get(url)
    try:
        src = result.content
        soup = BeautifulSoup(src, 'lxml')
        table = soup.find('table', {'class': 'sortable'})
        names = []
        trs = table.find_all('tr')
        rows = []
        for tr in trs:
            try:
                td = tr.find('td', {'data-stat': 'player'})
                a_tag = td.find('a')
                name = a_tag.get_text()
                name = unidecode.unidecode(name)
                name = re.sub(r'[^\w\s]','',name)
                names.append(name)
            except:
                pass
        for x in range(len(names)):
            names[x] = names[x].split()
            names[x].append(id_number)
            id_number += 1
        print(TeamID,"roster successfully parsed")
        return names
    except:
        return False



In [24]:
def nba_teams_list(year):
    team_id = 101
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}.html"
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'lxml')
    table = soup.find('table', {'class': 'stats_table sortable', 'data-cols-to-freeze': ',2', 'id': 'per_game-team'})
    teams = []
    ids = {}
    trs = table.find_all('tr')
    rows = []
    x = 0
    for tr in trs[1:]:
        #print(tr.prettify())
        try:
            td = tr.find('td', {'data-stat': 'team', 'class': 'left'})
            a_tag = td.find('a')
            team = a_tag.get_text()
            teams.append(team)
            link = a_tag.attrs['href']
            link = list(link)
            acronym = link[7] + link[8] + link[9]
            ids[teams[x]] = []
            ids[teams[x]].append(acronym)
            x+=1
        except:
            pass
    for x in range(len(teams)):
        ids[teams[x]].append(team_id)
        team_id+=1
    return teams, ids

In [25]:
nba_teams_list('2004')

(['Dallas Mavericks',
  'Sacramento Kings',
  'Los Angeles Lakers',
  'Milwaukee Bucks',
  'Denver Nuggets',
  'Seattle SuperSonics',
  'Memphis Grizzlies',
  'Boston Celtics',
  'Los Angeles Clippers',
  'Minnesota Timberwolves',
  'Phoenix Suns',
  'Orlando Magic',
  'Golden State Warriors',
  'Cleveland Cavaliers',
  'Atlanta Hawks',
  'New York Knicks',
  'Washington Wizards',
  'New Orleans Hornets',
  'San Antonio Spurs',
  'Indiana Pacers',
  'Portland Trail Blazers',
  'New Jersey Nets',
  'Miami Heat',
  'Detroit Pistons',
  'Houston Rockets',
  'Chicago Bulls',
  'Utah Jazz',
  'Philadelphia 76ers',
  'Toronto Raptors'],
 {'Dallas Mavericks': ['DAL', 101],
  'Sacramento Kings': ['SAC', 102],
  'Los Angeles Lakers': ['LAL', 103],
  'Milwaukee Bucks': ['MIL', 104],
  'Denver Nuggets': ['DEN', 105],
  'Seattle SuperSonics': ['SEA', 106],
  'Memphis Grizzlies': ['MEM', 107],
  'Boston Celtics': ['BOS', 108],
  'Los Angeles Clippers': ['LAC', 109],
  'Minnesota Timberwolves': ['MI

In [26]:
def generate_rosters(year):
    teamrosters = {}
    id_number = 1001
    teams, team_ids = nba_teams_list(year)
    for team in teams:
        teamrosters[team] = parse_roster(team_ids[team][0], year, id_number)
        id_number+=len(teamrosters[team])
    return teamrosters, team_ids

In [27]:
def generate_single_team_roster(team, year):
    roster = []
    id_number = 1001
    teams, team_ids = nba_teams_list(year)
    roster = parse_roster(team_ids[team][0], year, id_number)
    return roster, team_ids


In [28]:
def parse_player_stats(name, team, year):
    name = name.split()
    name.append('1001')
    teams, ids = nba_teams_list(year)
    team_id = ids[team][0]
    df = statsParser(name, year, '01', team_id)
    return df


In [29]:
def parse_entire_teams_stats(team, year):
    roster, team_ids = generate_single_team_roster(team, year)
    for x in range(len(roster)):
        try:
            if x == 0:
                player = roster[x]
                df = statsParser(player, year, '01', team_ids[team][0])
            else:
                player = roster[x]
                df = df.append(statsParser(player, year, '01', team_ids[team][0]))
        except:
            pass
    return df

In [30]:
def parse_entire_league_stats(year):
    rosters, team_ids = generate_rosters(year)
    y = 0
    for team in rosters:
        for x in range(len(rosters[team])):
            try:
                if x == 0 and y == 0:
                    player = rosters[team][x]
                    print(player)
                    df = statsParser(player, year, '01', team_ids[team][0])
                else:
                    player = rosters[team][x]
                    df = df.append(statsParser(player, year, '01', team_ids[team][0]))
            except:
                pass
        y+=1
    return df

In [31]:
def save_df(df, title):
    df.to_csv(rf'C:\Users\jleus\OneDrive\Documents\CSVS\{title}.csv', index=False)


In [32]:
generate_single_team_roster('Los Angeles Lakers', '2002')

LAL roster successfully parsed


([['Kobe', 'Bryant', 1001],
  ['Joe', 'Crispin', 1002],
  ['Derek', 'Fisher', 1003],
  ['Rick', 'Fox', 1004],
  ['Devean', 'George', 1005],
  ['Robert', 'Horry', 1006],
  ['Lindsey', 'Hunter', 1007],
  ['Mark', 'Madsen', 1008],
  ['Jelani', 'McCoy', 1009],
  ['Stanislav', 'Medvedenko', 1010],
  ['Shaquille', 'ONeal', 1011],
  ['Mike', 'Penberthy', 1012],
  ['Mitch', 'Richmond', 1013],
  ['Brian', 'Shaw', 1014],
  ['Samaki', 'Walker', 1015]],
 {'Dallas Mavericks': ['DAL', 101],
  'Sacramento Kings': ['SAC', 102],
  'Los Angeles Lakers': ['LAL', 103],
  'Orlando Magic': ['ORL', 104],
  'Minnesota Timberwolves': ['MIN', 105],
  'Golden State Warriors': ['GSW', 106],
  'Seattle SuperSonics': ['SEA', 107],
  'Milwaukee Bucks': ['MIL', 108],
  'Indiana Pacers': ['IND', 109],
  'San Antonio Spurs': ['SAS', 110],
  'Portland Trail Blazers': ['POR', 111],
  'Boston Celtics': ['BOS', 112],
  'New Jersey Nets': ['NJN', 113],
  'Utah Jazz': ['UTA', 114],
  'Los Angeles Clippers': ['LAC', 115],
  '

In [33]:
parse_entire_teams_stats('Los Angeles Lakers', '2002')

LAL roster successfully parsed
https://www.basketball-reference.com/players/B/BryanKo01/gamelog/2002/
200
https://www.basketball-reference.com/players/C/CrispJo01/gamelog/2002/
200
https://www.basketball-reference.com/players/F/FisheDe01/gamelog/2002/
200
https://www.basketball-reference.com/players/F/FoxRi01/gamelog/2002/
200
https://www.basketball-reference.com/players/G/GeorgDe01/gamelog/2002/
200
https://www.basketball-reference.com/players/H/HorryRo01/gamelog/2002/
200
https://www.basketball-reference.com/players/H/HunteLi01/gamelog/2002/
200
https://www.basketball-reference.com/players/M/MadseMa01/gamelog/2002/
200
https://www.basketball-reference.com/players/M/McCoyJe01/gamelog/2002/
200
https://www.basketball-reference.com/players/M/MedveSt01/gamelog/2002/
200
https://www.basketball-reference.com/players/O/ONealSh01/gamelog/2002/
200
https://www.basketball-reference.com/players/P/PenbeMi01/gamelog/2002/
200
https://www.basketball-reference.com/players/R/RichmMi01/gamelog/2002/


,G,Date,Age,Tm,Opp,GS,MP,FG,FGA,FG%,...,STL,BLK,TOV,PF,PTS,GmSc,+/-,,,PlayerID
1,1,2001-10-30,23-068,LAL,,POR,W (+11),1,42:36,11,...,7,4,3,1,2,0,29,23.2,+4,1001
2,2,2001-11-01,23-070,LAL,@,UTA,W (+4),1,43:03,11,...,6,8,3,1,2,3,39,35.1,+7,1001
3,3,2001-11-02,23-071,LAL,,PHO,W (+23),1,37:40,9,...,6,9,2,1,7,4,24,20.8,+19,1001
4,4,2001-11-04,23-073,LAL,,UTA,W (+4),1,44:17,13,...,6,7,3,1,5,5,38,31.1,+3,1001
5,5,2001-11-09,23-078,LAL,,MEM,W (+24),1,33:01,5,...,6,7,5,3,2,0,13,16.7,+29,1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,65,2002-04-09,26-043,LAL,,UTA,W (+30),1,23:23,2,...,10,3,0,1,1,4,4,7.8,-2,1015
69,66,2002-04-11,26-045,LAL,,MIN,W (+13),1,27:16,2,...,10,0,0,1,0,3,4,6.4,-2,1015
70,67,2002-04-14,26-048,LAL,@,POR,L (-8),1,21:10,3,...,2,2,1,0,1,3,7,5.1,-5,1015
71,68,2002-04-15,26-049,LAL,,SEA,W (+7),1,28:24,5,...,7,0,0,0,1,4,12,8.7,+10,1015


In [34]:
df1 = advancedStatsParser(['Lebron', 'James', '1001'], '2012', '01', 'MIA')

https://www.basketball-reference.com/players/J/JamesLe01/gamelog-advanced/2012/
200
             TS%   eFG%  ORB%  DRB%  TRB%  AST%  STL%  BLK%  TOV%  USG%  ORtg  \
1              1  36:16  .676  .579  10.4  21.1  16.1  33.9   2.7   4.9   9.9   
2              1  36:00  .724  .643   4.4  18.5  12.1  22.2   2.6   2.5  14.3   
3              1  38:49  .649  .565   0.0  18.5   8.9  43.4   4.0   3.9   6.9   
4              1  37:13  .612  .615   2.8  20.1  11.3  50.8   5.0   2.4  12.6   
5              1  28:02  .633  .600   0.0  29.1  17.7  22.7   1.7   0.0  13.7   
..           ...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
65             1  35:20  .580  .472   0.0  34.8  17.6  38.6   0.0   2.5   4.1   
66  Did Not Play   None  None  None  None  None  None  None  None  None  None   
67             1  35:52  .630  .548   2.8  24.7  12.6  33.0   1.5   4.8  16.4   
68  Did Not Play   None  None  None  None  None  None  None  None  None  None   
69  Did Not Play   None  

In [35]:
df2 = statsParser(['Lebron', 'James', '1001'], '2012', '01', 'MIA')

https://www.basketball-reference.com/players/J/JamesLe01/gamelog/2012/
200


In [36]:
concatStats(df2, df1)

ValueError: columns overlap but no suffix specified: Index(['GmSc', ''], dtype='object')